In [1]:
import random
import math
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
import scipy
from scipy.optimize import minimize
from scipy.stats import gamma, norm
import seaborn as sns
import time

from functools import partial

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [2]:
N = int(1e5)  # MC iterations
def _r(x): return np.round(x, 2)
def _CI(mu, std, n=1, z=1): return mu + (z/np.sqrt(n)) * np.array([-std, std])

# Parameter Estimation

## MLE for Gamma distribution with initialization values from MoMe

Let's find the MLE of Gamma parameters $\alpha$ (shape) and $\lambda$ (rate) starting from MoM estimators. We'll simulate 100 data points from a Gamma with $\alpha_0 = 3$ and $\lambda_0 = 0.5$

In [3]:
alpha = 3
lambd = 0.5
n = 100
X = gamma.rvs(alpha, scale=1/lambd, size=n)

The MoMe of a Gamma distribution are:
$$\hat{\lambda} = \frac{\bar{X}_n}{s^2_n}$$
$$\hat{\alpha} = \frac{\bar{X}_n^2}{s_n^2}$$
these will be our starting guesses for the MLE

In [4]:
alpha_mom = np.mean(X)**2 / np.var(X)
lambd_mom = np.mean(X) / np.var(X)
print('parameters estimated from MoM:')
print('alpha:', alpha_mom)
print('lambda:', lambd_mom)

parameters estimated from MoM:
alpha: 2.9192312119021797
lambda: 0.47378421404346205


We can see that the MoMe guesses are pretty close to the true paramters. \\
Let's write down the negative log likelhood and minimize it for w.r.t. $\alpha$ and $\lambda$

In [5]:
def gamma_neg_log_lik(params):
  alpha, lambd = params
  return -(n*alpha*np.log(lambd) + (alpha-1)*sum(np.log(X)) \
           - lambd*sum(X) - n*np.log(scipy.special.gamma(alpha)))

In [6]:
params = (alpha_mom, lambd_mom)
bnds = ((0, None), (0, None))
result = minimize(gamma_neg_log_lik, params, bounds=bnds, method='L-BFGS-B')
print(result.x)
print('Number of iterations:', result.nit)

[2.75332426 0.4468579 ]
Number of iterations: 5


We can see that this result is better than MoMe (very few times it's worse, we'll build CI to understand the variability better). Also, the number of iterations performed is very limited as we were expecting: starting from good guesses of the parameters, the algorithm converges quickly. \\
Let's now build confidence intervals for the estimated parameters through **parametric bootstrap** and compare MoMe and MLE.

In [7]:
N = int(1e3)
alpha = 3
lambd = 0.5
n = 100
bnds = ((0, None), (0, None))
MoMe = []  # will contain deviations
MLE = []
for _ in range(N):
  X = gamma.rvs(alpha, scale=1/lambd, size=n)
  # MoMe
  alpha_mom = np.mean(X)**2 / np.var(X)
  lambd_mom = np.mean(X) / np.var(X)
  # directly computing the deviation from true params here
  MoMe.append((alpha_mom-alpha, lambd_mom-lambd))
  # MLE
  result = minimize(gamma_neg_log_lik, (alpha_mom, lambd_mom), 
                    bounds=bnds, method='L-BFGS-B')
  alpha_mle, lambd_mle = result.x
  MLE.append((alpha_mle-alpha, lambd_mle-lambd))

In [8]:
CI = lambda x, dev: x + np.array([np.percentile(dev, 5),
                                  np.percentile(dev, 95)])
# MoMe and MLE deviations from true paramter
alpha_mom_dev, lambd_mom_dev = zip(*MoMe)
alpha_mle_dev, lambd_mle_dev = zip(*MLE)
print('90% CI:')
print('alpha (MoMe):', CI(alpha, alpha_mom_dev))
print('lambda (MoMe):', CI(lambd, lambd_mom_dev))
print('alpha (MLE):', CI(alpha, alpha_mle_dev))
print('lambda (MLE):', CI(lambd, lambd_mle_dev))

90% CI:
alpha (MoMe): [2.37836845 4.00967412]
lambda (MoMe): [0.39135354 0.68556171]
alpha (MLE): [2.46183098 3.83111037]
lambda (MLE): [0.39930414 0.64965277]


We can clearly see that the MLE estimates have lower variance than the MoMe as the CI are much narrower! \\
This is in line with the fact that MLE maximizes **statistical efficiency**, i.e. the MSE of the estimator $E_{\theta_0}(\hat{\theta}_n - \theta_0)^2$ is minimized when $\hat{\theta}_n$ is chosen to be the MLE.

## Plug-in estimators

Suppose that we wish to estimate the $p^{th}$ quantile $q_0$ of an exponentially distributed population, in the presence of a random sample $X_1, X_2, \dots , X_n$. \\
For an exponential population with parameter $\lambda$, we have $q = -\frac{1}{\lambda} log(1 − p)$. \\
We know that the MLE for $\lambda_0$ is: $\hat{\lambda} = \frac{1}{\overline{X}_n}$. \\
Thus, according to the plug-in principle, the most statistically efficient way to estimate the quantile is to plug-in the MLE in the quantile equation.

In [9]:
lambd = 2
n = 1000
X = np.random.exponential(scale=1/lambd, size=n)

In [10]:
lambd_mle = 1 / np.mean(X)
print('Lambda from MLE:', lambd_mle)

Lambda from MLE: 2.077998166798869


In [11]:
p = 0.25
q_hat = lambda lambd, p: -(1/lambd) * np.log(1-p)
print('Estimated quantile through MLE plug-in:', q_hat(lambd_mle, p))
print('Observed quantile from data:', np.percentile(X, p*100))

Estimated quantile through MLE plug-in: 0.13844192793247342
Observed quantile from data: 0.13020970230653706


We now want to construct large-sample CI for the true quantile $q_0$. We know that:
$$\sqrt{n}(\hat{q}_n - q_0) \Rightarrow \sigma log(1-p)^{-1}N(0,1)$$
where $\sigma^2 = Var_{\theta_0}(X_1)$; thus we need the the variance of the X's. Firstly, we might think about estimating it through the sample variance, but we also know that for the exponential distribution
$$Var(X) = \frac{1}{\lambda^2}$$
so by plug-in we have that:
$$\hat{Var(X)} = \overline{X}^2$$
Let's compare the two

In [12]:
print('Sample variance:', np.var(X))
print('Plug-in variance:', np.mean(X)**2)

Sample variance: 0.23424834092638047
Plug-in variance: 0.23158460196519076


So our 95% CI is given by:

In [13]:
z = 1.96
CI = np.mean(X)*np.log((1-p)**-1) * np.array([1-z/np.sqrt(n), 1+z/np.sqrt(n)])
print('95% Confidence Interval for lambda:', CI)

95% Confidence Interval for lambda: [0.12986121 0.14702265]


Let's check that this is really 95% through MC simulation:

In [14]:
lambd = 2
n = 1000
p = 0.25
q_hat = lambda lambd, p: -(1/lambd) * np.log(1-p)
N = int(1e3)
Z = np.zeros(N)
for i in range(N):
  X = np.random.exponential(scale=1/lambd, size=n)
  lambd_mle = 1 / np.mean(X)
  q_MC = q_hat(lambd_mle, p)
  if CI[0] <= q_MC <= CI[1]:
    Z[i] = 1
print('Observed probability that q_MC is contained in the CI:', np.mean(Z))

Observed probability that q_MC is contained in the CI: 0.75
